In [ ]:
# nuclio: start-code

In [ ]:
import sys, os
import ast
import json
module_paths = ast.literal_eval(os.getenv('MODULE_PATHS', None))
if module_paths != None:
    for module_path in module_paths:
        sys.path.append(module_path)
        
import_modules = ast.literal_eval(os.getenv('IMPORT_MODULES', None))
if import_modules != None:
    imports={}
    import importlib
    for  module in import_modules:
        imports[module] = importlib.import_module(module)

In [2]:
import ast
import json
import v3io.dataplane

In [3]:
def output_stream(context,message,details):
    Records=[]
    # message must be a Python dictionary
    record = json.dumps(message).encode('utf-8')
    Records.append({
            "data":  record,
            "partition_key": message['PartitionKey'],
            })
    context.user_data.v3io_client.put_records(container=os.getenv('OUTPUT_STREAM_CONTAINER'),
                                             path=details['output_stream'],records=Records)
    return

In [4]:
def igz_stream_init(context):
    setattr(context.user_data,'v3io_client',
            v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', 
                                  access_key=os.getenv("V3IO_ACCESS_KEY")))


In [5]:
def output_function(context,message,details):
    call_function = "%s(context,message)"% details['post_process_function']
    eval(call_fucntion)
    return

In [7]:
def output_http(context,message,details):
    print("If I was working I would post to %s the key %s"% (details['url'],message['PartitionKey']))
    return

In [12]:
def post_process(context,message):
    for output in _context.user_data.outputs:
        if 'condition' in output:
            try:
                if not eval(output['condition']):
                    continue
            except:
                context.logger.debug("Invalid condition spec %s"% output['condition'])
                
        call_function = "output_%s(context,message,output)"%output['kind']
        eval(call_function)

In [13]:
def process(context,message):
    # message is dictionary loaded from the stream event
    _function="context.user_data.processing_class.%s(context,message)"%os.getenv('PROCESSING_FUNCTION')
    post_message = eval(_function)
    post_process(context,post_message)
    return

In [15]:
def init_context(context):
    # IMPORTANTE: Load the model here
    # This code will put the model in the context of the function
    _module, _class = os.getenv('CLASS_LOAD_FUNCTION').split(".")
    _load_class = "imports['%s'].%s()"%(_module, _class)
    setattr(context.user_data,'processing_class',eval(_load_class))
    setattr(context.user_data,'outputs',ast.literal_eval(os.getenv('OUTPUTS')))
    _stream_output = False
    for output in context.user_data.outputs:
        if output['kind'] == 'stream':
            _stream_output = True
            
    if _stream_output:
        igz_stream_init(context)

    return
    

In [11]:
def handler(context, event):
    context.logger.debug(event.body)
    try:
        message = ast.literal_eval(event.body.decode('utf-8'))
    except:
        print("Malformated Json ", event.body)
    message['shard']= event.shard_id
    process(context,message)
      
    return 

In [ ]:
# nuclio: end-code